In [ ]:
import numpy as np
import yaml
from tqdm.notebook import tqdm
import galsim
import batoid
import wfsim
import matplotlib.pyplot as plt

In [ ]:
# Some initial setup
# We'll do r-band for this demo.
bandpass = galsim.Bandpass("LSST_r.dat", wave_type='nm')
telescope = batoid.Optic.fromYaml("LSST_r.yaml")
pixel_scale = 10e-6

In [ ]:
bandpass.effective_wavelength

In [ ]:
# Setup observation parameters.  Making ~plausible stuff up.
observation = {
    'zenith': 30 * galsim.degrees,
    'raw_seeing': 0.7 * galsim.arcsec,  # zenith 500nm seeing
    'wavelength': bandpass.effective_wavelength,
    'exptime': 15.0,  # seconds
    'temperature': 293.,  # Kelvin
    'pressure': 69.,  #kPa
    'H2O_pressure': 1.0  #kPa
}

In [ ]:
# Setup atmospheric parameters
atm_kwargs = {
    'screen_size': 819.2,
    'screen_scale': 0.1,
    'nproc': 6  # create screens in parallel using this many CPUs
}

In [ ]:
# We loaded the fiducial telescope, but we actually want to perturb it 
# out of alignment a bit and misfigure the mirrors a bit.
# The big question is how much to perturb each potential 
# degree-of-freedom.  Let's not dwell on that at the moment though; for
# demonstration, the following will do.

rng = np.random.default_rng()

# Misalignments of M2 and camera first
M2_offset = np.array([
    rng.uniform(-0.0001, 0.0001),  # meters
    rng.uniform(-0.0001, 0.0001),
    rng.uniform(-0.00001, 0.00001),
])    
M2_tilt = (
    batoid.RotX(np.deg2rad(rng.uniform(-0.01, 0.01)/60)) @
    batoid.RotY(np.deg2rad(rng.uniform(-0.01, 0.01)/60))
)

camera_offset = np.array([
    rng.uniform(-0.001, 0.001),  # meters
    rng.uniform(-0.001, 0.001),
    rng.uniform(-0.00001, 0.00001),    
])
camera_tilt = (
    batoid.RotX(np.deg2rad(rng.uniform(-0.01, 0.01)/60)) @
    batoid.RotY(np.deg2rad(rng.uniform(-0.01, 0.01)/60))
)

perturbed = (
    telescope
    .withGloballyShiftedOptic("M2", M2_offset)
    .withLocallyRotatedOptic("M2", M2_tilt)
    .withGloballyShiftedOptic("LSSTCamera", camera_offset)
    .withLocallyRotatedOptic("LSSTCamera", camera_tilt)
)

# Now let's perturb the mirrors, we should use the actual mirror modes
# here, but for now we'll just use Zernike polynomials.  

M1M3_modes = rng.uniform(-0.05, 0.05, size=25) # waves
M1M3_modes *= bandpass.effective_wavelength*1e-9 # -> meters
# M1M3 bends coherently, so use a single Zernike perturbation for both,
# Set the outer radius to the M1 radius so the polynomial doesn't 
# explode.  It's fine to use a circular Zernike here though (no inner 
# radius).

M1M3_surface_perturbation = batoid.Zernike(
    M1M3_modes,
    R_outer=telescope['M1'].obscuration.original.outer,
)
perturbed = perturbed.withSurface(
    "M1",
    batoid.Sum([
        telescope['M1'].surface,
        M1M3_surface_perturbation
    ])
)
perturbed = perturbed.withSurface(
    "M3",
    batoid.Sum([
        telescope['M3'].surface,
        M1M3_surface_perturbation
    ])
)

# M2 gets independent perturbations from M1M3
M2_modes = rng.uniform(-0.05, 0.05, size=25) # waves
M2_modes *= bandpass.effective_wavelength*1e-9 # -> meters

M2_surface_perturbation = batoid.Zernike(
    M2_modes,
    R_outer=telescope['M2'].obscuration.original.outer,
)
perturbed = perturbed.withSurface(
    "M2",
    batoid.Sum([
        telescope['M2'].surface,
        M2_surface_perturbation
    ])
)

In [ ]:
# We can take a quick look at how we've perturbed the optics by making 
# a spot diagram.  The batoid.spot tool returns points in meters, so
# we divide by pixel_scale to get pixels.  We also look in a few points
# around the field of view to get a global picture.

for thx, thy in [(0,0), (-1.5, 0), (1.5, 0), (0, -1.5), (0, 1.5)]:
    sx, sy = batoid.spot(
        perturbed, 
        np.deg2rad(thx), np.deg2rad(thy), 
        bandpass.effective_wavelength*1e-9, 
        nx=128
    )
    plt.figure()
    plt.scatter(sx/pixel_scale, sy/pixel_scale, s=1)
    plt.show()

In [ ]:
# To make donuts, we need to be intra-focal or extra-focal.
# To simulate normal science operations mode, shift the detector:
intra = perturbed.withGloballyShiftedOptic(
    "Detector", [0, 0, -0.0015]
)
extra = perturbed.withGloballyShiftedOptic(
    "Detector", [0, 0, +0.0015]
)

In [ ]:
intra_simulator = wfsim.SimpleSimulator(
    observation,
    atm_kwargs,
    intra,
    bandpass,
    shape=(512, 512),
    rng=rng
)

In [ ]:
# Now we can choose some parameters for a star and start simulating
# First, choose a field angle.  At the moment, the simulator code only 
# works close to the boresight direction, so just use that.  I'll 
# extend that soon.
thx = np.deg2rad(0.0)
thy = np.deg2rad(0.0)

In [ ]:
# We also want to simulate chromatically.  We could fetch an actual 
# stellar SED for this, but it's easier and probably always good enough
# to just use a black body with a reasonable temperature.
star_T = rng.uniform(4000, 10000)
sed = wfsim.BBSED(star_T)

In [ ]:
# We also need a flux (which needs to be an integer):
flux = int(rng.uniform(1_000_000, 2_000_000))

In [ ]:
intra_simulator.add_star(thx, thy, sed, flux, rng)

In [ ]:
# We can look at our star now:
plt.figure()
plt.imshow(intra_simulator.image.array)
plt.show()

In [ ]:
# Our image doesn't have any sky background noise in it yet.  
# Here we add some.
intra_simulator.add_background(1000.0, rng)

In [ ]:
# Here's our final star
plt.figure()
plt.imshow(intra_simulator.image.array)
plt.show()

In [ ]:
# Finally, what were the actual Zernike's for the perturbed telescope
# we generated?  Get that using batoid.zernike:
zs = batoid.zernike(
    perturbed, 
    thx, thy, 
    bandpass.effective_wavelength*1e-9  # batoid wants meters,
)
zs *= bandpass.effective_wavelength # waves -> nm
for j in range(4, 23):
    print(f"{j:>2d}  {zs[j]:6.1f} nm")

In [ ]:
# TODO items:
# - let image cover non-central regions of the focal plane
# - allow easy reuse of the generated atmosphere, but with different
#   perturbed telescopes
# - allow easy resetting of the accumulated image
# - get chip coords from obs_lsst?
# - add tech to use phase screen with target Zernikes in front of 
#   telescope